<a href="https://colab.research.google.com/github/mahault/Quantum-C/blob/main/Election_probability_QNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pennylane

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.1/56.1 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 930.0/930.0 kB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 47.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 53.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 57.4 MB/s eta 0:00:00


In [3]:
import pennylane as qml
from pennylane import numpy as np

# Quantum device
n_qubits = 2
dev = qml.device("default.qubit", wires=n_qubits)

# Quantum Neural Network
@qml.qnode(dev)
def qnn(params, features):
    """
    QNN for election prediction.
    - params: Trainable parameters
    - features: Encoded election features
    """
    # Encode features into quantum states
    qml.RY(features[0], wires=0)  # Turnout (amplitude rotation)
    qml.RZ(features[1], wires=0)  # Polling margin (phase rotation)
    qml.RY(features[2], wires=1)  # Unemployment rate (amplitude rotation)

    # Apply trainable variational layer
    qml.RY(params[0], wires=0)
    qml.RZ(params[1], wires=0)
    qml.CNOT(wires=[0, 1])  # Entanglement
    qml.RY(params[2], wires=1)
    qml.RZ(params[3], wires=1)

    # Return probabilities for each class
    return qml.probs(wires=1)

# Data preprocessing
def normalize(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data)) * np.pi  # Map to [0, pi]

# Example dataset
features = np.array([
    [0.65, -0.15, 0.04],  # California
    [0.58, 0.12, 0.05],   # Texas
    [0.62, 0.03, 0.06],   # Florida
    [0.67, -0.18, 0.04],  # New York
    [0.63, -0.05, 0.05],  # Pennsylvania
])
labels = np.array([-1, 1, 1, -1, -1])  # Democrats: -1, Republicans: +1

# Normalize features
normalized_features = np.array([normalize(row) for row in features])

# Initialize random parameters for the QNN
params = np.random.uniform(0, np.pi, size=4)

# Loss function
def loss(params, features, labels):
    total_loss = 0
    for i in range(len(features)):
        probs = qnn(params, features[i])
        target = (1 + labels[i]) // 2  # Convert -1/1 to 0/1
        total_loss -= np.log(probs[target])  # Cross-entropy loss
    return total_loss / len(features)

# Training loop
opt = qml.GradientDescentOptimizer(stepsize=0.1)
epochs = 50

for epoch in range(epochs):
    params = opt.step(lambda p: loss(p, normalized_features, labels), params)
    if epoch % 10 == 0:
        print(f"Epoch {epoch}: Loss = {loss(params, normalized_features, labels):.4f}")

# Testing on a new state
new_state = normalize(np.array([0.61, 0.02, 0.05]))  # New state data
probs = qnn(params, new_state)
predicted_label = 1 if probs[1] > probs[0] else -1
print(f"Predicted probabilities: {probs}")
print(f"Predicted winner: {'Republicans' if predicted_label == 1 else 'Democrats'}")


Epoch 0: Loss = 1.1745
Epoch 10: Loss = 0.5914
Epoch 20: Loss = 0.4983
Epoch 30: Loss = 0.4655
Epoch 40: Loss = 0.4506
Predicted probabilities: [0.49413897 0.50586103]
Predicted winner: Republicans
